In [1]:
import time
import re
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By # 언제까지 브라우저 접속을 기다려줄 지 결정
from selenium.webdriver.support.ui import WebDriverWait # By 모듈과 같이 자주 쓰인다.
from selenium.webdriver.support import expected_conditions as EC # 어떤 상태까지 기다리겠다(페이지 클릭이나 커서 이동)
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

from tqdm import tqdm

# 찾을 정보
기업이름,  마감날짜, 필요조건, 우대조건,  연봉정보, 주요 업무, 신입경력, 

In [6]:
def get_contents_dict(soup):
    
    # initialization
    title = np.nan
    companyname = np.nan
    duedate = np.nan
    job = np.nan
    salary = np.nan
    exp = np.nan
    maintask = np.nan
    requirement = np.nan
    prefered = np.nan
    location = np.nan
    content_list = []

    # 공고 제목
    title = soup.find_all('h1', class_ = 'ttl')[0].text.strip()
    # print(title)

    # 마감 날짜
    try: duedate = soup.find_all('li', class_ = 'calendar')[0].find_all('span',class_ = 'day_text')[-1].text.strip()
    except: duedate = soup.find_all('li', class_ = 'calendar')[0].find_all('span')[-1].text.strip()
    print(duedate)

    # 기업 이름
    companyname = soup.find_all('span', class_ = 'company_name')[0].text.strip()
    # print(companyname)

    # 직무
    job = soup.find('li', class_ = 'briefcase').find('span',class_ = 'cont').text.strip()
    # print(briefcase)

    # 급여
    salary = soup.find('li', class_ = 'money').find('span',class_ = 'cont').text.strip()
    # print(salary)

    # 신입/경력
    exp = soup.find('li', class_ = 'experience').find('span',class_ = 'cont').text.strip()
    # print(exp)

    # 주요 업무, 자격요건, 우대사항
    paragraph_all = soup.find_all('div', class_ = 'paragraph')
    for i in paragraph_all:
        if '주요업무' in i.text.strip():
            maintask = i.find('div', class_ = 'text').text.strip()
            # print('주요업무: ', maintask)

        elif '자격요건' in i.text.strip():
            requirement = i.find('div', class_ = 'text').text.strip()
            # print('자격요건:', requirement)

        elif '우대사항' in i.text.strip():
            prefered = i.find('div', class_ = 'text').text.strip()
            # print('우대사항:', prefered)
            
    # 위치
    for i in soup.find_all('div', class_ = 'line_text_list'):
        if '회사위치' in i.text.strip():
            location = i.find('p').text.strip()
            # print('위치:', location)

    return {'companyname': companyname, 'duedate': duedate, 'job': job, 'salary': salary, 'exp': exp, 'maintask': maintask, 'requirement': requirement, 'prefered': prefered, 'location': location}


# 스크래핑 시작

In [9]:
df = pd.DataFrame()

options = Options()
# options.add_argument("user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")
# chrome_options.add_argument("--headless") # 브라우저가 화면에 뜨지 않고 내부적으로 실행
driver = webdriver.Chrome("./chromedriver_85.exe", options = options)

# 잡플래닛 URL 접속 및 페이지 넘김
for i in range(1, 100):
    print('===================PAGE{}=================='.format(i))
    driver.get('https://www.jobplanet.co.kr/job/search?q=%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D&page={}'.format(i))

    # 크롬 브라우저 내부 대기
    driver.implicitly_wait(5)

    #하나의 페이지 안에서 하나의 공고 클릭
    for r in tqdm(range(1,11)):
        # print('========{}번째 공고========'.format(r))

        # Click each job decscription
        driver.find_element_by_xpath('//*[@id="job_search_app"]/div/div[2]/div[1]/div[2]/ul/li[{}]/a'.format(r)).click()
        time.sleep(1)

        # Refresh source
        bs = BeautifulSoup(driver.page_source, 'html.parser')

        # Get contents
        dict_ = get_contents_dict(bs)
        df = df.append(dict_, ignore_index=True)

===================PAGE1==================

 10%|█         | 1/10 [00:01<00:11,  1.32s/it]2020. 11. 13

 20%|██        | 2/10 [00:02<00:10,  1.27s/it]2020. 10. 09

 30%|███       | 3/10 [00:03<00:08,  1.24s/it]2020. 09. 30

 40%|████      | 4/10 [00:04<00:07,  1.23s/it]2020. 09. 30

 50%|█████     | 5/10 [00:06<00:06,  1.21s/it]2020. 12. 31

 60%|██████    | 6/10 [00:07<00:04,  1.20s/it]2020. 09. 30

 70%|███████   | 7/10 [00:08<00:03,  1.19s/it]2020. 12. 31

 80%|████████  | 8/10 [00:09<00:02,  1.17s/it]2020. 10. 24

 90%|█████████ | 9/10 [00:10<00:01,  1.19s/it]2020. 11. 21

100%|██████████| 10/10 [00:11<00:00,  1.19s/it]2021. 03. 04
===================PAGE2==================


 10%|█         | 1/10 [00:01<00:12,  1.43s/it]상시채용

 20%|██        | 2/10 [00:02<00:10,  1.36s/it]2020. 10. 05

 30%|███       | 3/10 [00:03<00:09,  1.31s/it]2020. 10. 04

 40%|████      | 4/10 [00:04<00:07,  1.26s/it]2020. 10. 05

 50%|█████     | 5/10 [00:06<00:06,  1.24s/it]2020. 09. 30

 60%|██████    | 6/

KeyboardInterrupt: 

In [8]:
df

,companyname,duedate,exp,job,location,maintask,prefered,requirement,salary
0,(주)코드브릭,2020. 11. 13,3년 이상,데이터분석,서울 강남구 역삼로 169 팁스타운 S2 명우빌딩 7층,- 데이터 기반으로 한 가설 설정과 효율적인 실험 설계 및 실행 (A/B Test ...,- Ecommerce Industry에 대한 이해가 높은 분\n- Lean Star...,"- GA, Amplitude 등의 트래킹 및 데이터 분석 관련 도구들을 사용하거나 ...",회사내규에 따름
1,(주)버킷플레이스,2020. 10. 09,3년 이상,"마케팅, 시장조사/분석","서울 서초구 서초대로 398, 플래티넘타워 19층","- 마케팅 주요 지표를 정의/관리하며, 운영에 도움을 줄 수 있는 데이터를 리포팅 ...",- 커머스 비즈니스/스타트업 유경험자\n- 디지털 광고 매체 및 3rd party ...,- 데이터 분석 경력 최소 3년 이상의 관련 업무 경력\n- 원하는 데이터를 얻기 ...,회사내규에 따름
2,(주)스마일게이트홀딩스,2020. 09. 30,8년 이상,"데이터분석, 경영기획/전략, 시장조사/분석","경기 성남시 분당구 판교역로 220, 5층",- 그룹사 전 게임 대상의 데이터 분석 및 인사이트 도출\n- 그룹사 게임 지표 분...,- 게임/IT 업계 내 유관 업무경험이 있으신 분,"- 경영정보 관리 및 분석 업무경험 8년 이상 (온라인, 모바일, 콘솔 게임/서비스...",회사내규에 따름
3,라인프렌즈(주),2020. 09. 30,경력무관,데이터분석,04405서울시 용산구 이태원로 200 라인프렌즈,- LINE Financial의 다양한 금융 데이터 분석\n- LINE 데이터를 포...,- 국가 유공자 및 장애인 등 취업보호대상자는 관계법령에 따라 우대합니다.\n- S...,"- 데이터 분석 프로그램(SQL, Python, SAS 등)을 활용한 데이터 분석 ...",회사내규에 따름
4,스포카(주),2020. 12. 31,3년 이상,데이터분석,서울 강남구 테헤란로 127 하나금융센터 강남사옥 4층 스포카,• 데이터 분석과 인사이트 제공: 스포카는 지난 9년간 쌓은 2500만 고객의 포인...,"• 1가지 이상의 프로그래밍 언어를 구사하시는 분: Python, R 등의 프로그래...",• 3년 이상의 데이터 분석 경력을 가지신 분: 데이터 분석가로서 3년 이상의 경력...,회사내규에 따름
5,(주)비아이매트릭스,2020. 09. 30,7년 이상,"데이터분석, 프로젝트매니저, 컨설팅","서울 강남구 선릉로 433, 신관 17층 (역삼동,세방빌딩)",- 당사 데이터통합분석 솔루션 i-STREAM 연구 개발 업무,"- 텐서플로우, AI, Machine learning 우대\n- 데이터 분석, 통계...","- 4년제 대졸 이상\n- 개발 경력 7년 이상 경력자\n- python, Pand...",면접 후 결정
6,(주)카카오엔터프라이즈,2020. 12. 31,경력무관,"데이터분석, 소프트웨어엔지니어, 웹개발","경기 성남시 분당구 판교역로 235, 3층",다음과 같은 업무를 하며 입사 후 업무 전환도 가능하니 1개 이상의 업무에 적합한 ...,"- 4년 이상의 분산 시스템 경험\n- kubernetes, docker 를 이용한...","- 전산/컴퓨터공학 전공\n- 탄탄한 관련 기본 지식 (운영체제, 알고리즘, 자료구...",회사내규에 따름
7,(주)커스토머인사이트,2020. 10. 24,"신입, 경력무관",데이터분석,서울 송파구 송파대로 167 테라타워 A720,"- 정형 데이터 분석, 비정형 데이터 분석\n- 빅 데이터 솔루션 Hadoop 구현",- Open Source Hadoop Data Platform 경험자\n- Micr...,- 경력 : 신입 또는 경력 2 년 이상\n- 대학 졸업 또는 석사 졸업 이상\n-...,회사내규에 따름
8,이씨이십일(주),2020. 11. 21,"신입, 경력무관",데이터분석,"서울특별시 강남구 삼성동 무역센터 트레이드타워 5F, 8F, 11F",- 해외시장정보수집\n- 트렌드 분석\n- 보고서 작성,- 보훈 및 장애 대상자는 관계법령에 의거하여 우대합니다.\n- 어문계열 전공자 및...,- 신입/경력,회사내규에 따름
9,(주)피플펀드컴퍼니,2021. 03. 04,"신입, 경력무관",데이터분석,서울특별시 강남구 테헤란로 518 12층 (대치동),"피플펀드의 데이터 분석가는 데이터를 가공하여 비즈니스 활용 가능한 모델을 개발하고,...","• 석사학위 이상 보유자\n• 빅데이터 기반 모델링 경험 보유자\n• 머신러닝, 딥...",• 신입 혹은 관련분야 경력 2~3년 미만\n• 상경계열 / 통계 / 수학 / 산업...,회사내규에 따름
